In [2]:
import s3fs
from mvpa2.datasets.mri import fmri_dataset
from mvpa2.clfs.meta import SplitClassifier
from mvpa2.generators.partition import NFoldPartitioner

import numpy as np
from mvpa2 import cfg

from mvpa2.clfs.svm import libsvm

import pandas as pd

clf = libsvm.SVM()

sclf = SplitClassifier(clf, NFoldPartitioner(cvtype=1), enable_ca=['stats'])
cv_sensana = sclf.get_sensitivity_analyzer()

fs = s3fs.S3FileSystem(anon=True)
ll = fs.ls('natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR')

betaslist = [l for l in ll if l.split('/')[6].startswith("betas") and l.endswith("nii.gz")]
fname = betaslist[0] # which session?

fs.get(fname, "tmp.nii.gz")

bt = fmri_dataset("tmp.nii.gz", mask = "shared/NSD/TPJ.nii")

print("Dataset shape is: ")
print(bt.shape)

# read from annotations csv file using pandas:
annot_file = pd.read_csv("shared/NSD/TPJ.nii")
rsp = df.response

# fds.sa['chnks'] = chunks_labels
# fds.sa['trgts'] = targets_labels

# zscore(fds, chunks_attr='chnks')

# sens = cv_sensana(evds)
# sens_comb = sens.get_mapped(
#     maxofabs_sample())
# nimg = map2nifti(fds, sens_comb)
# nimg.to_filename(os.path.join(args.output_dir, subj_name,
#                               subj_name + '_task-' + args.task + '_' + ROI_name +
#                               _' + 'all' + '_pattern.nii.gz'))

natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session01.nii.gz


/srv/conda/envs/notebook/lib/python3.7/site-packages/mvpa2/datasets/mri.py:78: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data, header = img.get_data(), img.header


FileNotFoundError: No such file or no access: 'shared/NSD/TPJ.nii'

In [ ]:
# results = []
# for fname in ll: 
    # fs.get(fname, "tmp.nii.gz")
    # img = nib.load("tmp.nii.gz")
    # XXX do something with one file
    
    
    # Append into total results:
    # results.append(XXX)